In [1]:
# Import libraries for SQL workings
import pandas as pd
import sqlite3

In [6]:
# Load our pre-made dataset (2_data_wrangling.csv)
df = pd.read_csv('2_data_wrangling.csv')
print("Dataset Loaded successfully!")

Dataset Loaded successfully!


In [7]:
#Preview of top 5 rows to check loaded correctly
df.head(5)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,6,2010-06-04T18:45:00.000Z,Falcon 9,NaN,LEO,CCSFS SLC 40,NaN,1,False,False,False,NaN,1,0,B0003,-80.577366,28.561857,0
1,7,2010-12-08T15:43:00.000Z,Falcon 9,NaN,LEO,CCSFS SLC 40,NaN,1,False,False,False,NaN,1,0,B0004,-80.577366,28.561857,0
2,8,2012-05-22T07:44:00.000Z,Falcon 9,525.0,LEO,CCSFS SLC 40,NaN,1,False,False,False,NaN,1,0,B0005,-80.577366,28.561857,0
3,9,2012-10-08T00:35:00.000Z,Falcon 9,400.0,ISS,CCSFS SLC 40,NaN,1,False,False,False,NaN,1,0,B0006,-80.577366,28.561857,0
4,10,2013-03-01T19:10:00.000Z,Falcon 9,677.0,ISS,CCSFS SLC 40,NaN,1,False,False,False,NaN,1,0,B0007,-80.577366,28.561857,0


In [8]:
# Create the SQL database
con = sqlite3.connect(":memory:")

# Convert the dataframe to a SQL table named: 'SPACEX'
df.to_sql("SPACEX", con, if_exists='replace', index=False, method="multi")
print("Database created and table 'SPACEX' now has data")

Database created and table 'SPACEX' now has data


In [10]:
# Print top 5 rows of the new table to check row data and columns
verification_query = 'SELECT * FROM SPACEX LIMIT 5'

print(pd.read_sql(verification_query, con))

   FlightNumber                      Date BoosterVersion  PayloadMass Orbit  \
0             6  2010-06-04T18:45:00.000Z       Falcon 9          NaN   LEO   
1             7  2010-12-08T15:43:00.000Z       Falcon 9          NaN   LEO   
2             8  2012-05-22T07:44:00.000Z       Falcon 9        525.0   LEO   
3             9  2012-10-08T00:35:00.000Z       Falcon 9        400.0   ISS   
4            10  2013-03-01T19:10:00.000Z       Falcon 9        677.0   ISS   

     LaunchSite Outcome  Flights  GridFins  Reused  Legs LandingPad  Block  \
0  CCSFS SLC 40    None        1         0       0     0       None      1   
1  CCSFS SLC 40    None        1         0       0     0       None      1   
2  CCSFS SLC 40    None        1         0       0     0       None      1   
3  CCSFS SLC 40    None        1         0       0     0       None      1   
4  CCSFS SLC 40    None        1         0       0     0       None      1   

   ReusedCount Serial  Longitude   Latitude  Class  
0  

In [13]:
# Define the query function
def sql_query(query):
    return pd.read_sql(query, con)

In [14]:
# Ready for queries!
print("Database is now ready for queries!")

Database is now ready for queries!


In [15]:
# Query 1 - Distinct Launchsites
query = """ 
SELECT DISTINCT LaunchSite FROM SPACEX
"""
sql_query(query)

,LaunchSite
0,CCSFS SLC 40
1,VAFB SLC 4E
2,KSC LC 39A


In [16]:
# Query 2 -  Count flights and successes per booster version
query = """
SELECT BoosterVersion, 
       COUNT(*) as Total_Flights, 
       SUM(Class) as Successful_Landings
FROM SPACEX
GROUP BY BoosterVersion
ORDER BY Total_Flights DESC
LIMIT 10
"""
sql_query(query)

,BoosterVersion,Total_Flights,Successful_Landings
0,Falcon 9,178,141


In [17]:
# Query 3 -  Calculate success rate per Orbit type
query = """
SELECT Orbit, 
       COUNT(*) as Flights, 
       SUM(Class) as Successes,
       (CAST(SUM(Class) AS FLOAT) / COUNT(*)) * 100 as Success_Rate
FROM SPACEX
GROUP BY Orbit
ORDER BY Success_Rate DESC
"""
sql_query(query)

,Orbit,Flights,Successes,Success_Rate
0,TLI,2,2,100.000000
1,HEO,1,1,100.000000
2,GEO,2,2,100.000000
3,ES-L1,1,1,100.000000
4,None,1,1,100.000000
5,VLEO,57,54,94.736842
6,SSO,12,11,91.666667
7,MEO,5,4,80.000000
8,LEO,15,12,80.000000
9,ISS,33,24,72.727273


In [18]:
# Query 4 -  Average Payload Mass for Success (1) vs Fail (0)
query = """
SELECT Class, AVG(PayloadMass) as Avg_Payload_kg
FROM SPACEX
GROUP BY Class
"""
sql_query(query)

,Class,Avg_Payload_kg
0,0,4615.613235
1,1,9059.096694


In [19]:
# Close the connection as standard practice!
con.close()
print("Database connection closed successfully.")

Database connection closed successfully.
